In [1]:
import numpy as np
from qiskit import transpile
import scipy.linalg as sl
import numpy.linalg as nl
from lchs import *
from lcu import *
from qiskit import QuantumCircuit,QuantumRegister
from qiskit.quantum_info import Operator, Statevector
np.set_printoptions(precision=8, suppress=True)

In [24]:
# def normalize_state(state_vector):
#     norm = np.linalg.norm(state_vector)
#     if norm == 0:
#         raise ValueError("State vector cannot be the zero vector.")
#     return state_vector / norm

# def state_preparation_circuit(state_vector):
#     """
#     Constructs a quantum circuit to prepare the given state vector.
    
#     Args:
#         state_vector (array_like): The desired state vector (must be normalized).
        
#     Returns:
#         QuantumCircuit: The circuit that prepares the state.
#     """
#     state_vector = np.array(state_vector, dtype=complex)
#     state_vector = normalize_state(state_vector)
#     num_qubits = int(np.log2(len(state_vector)))
#     if 2**num_qubits != len(state_vector):
#         raise ValueError("Length of state vector must be a power of 2.")
    
#     qr = QuantumRegister(num_qubits)
#     qc = QuantumCircuit(qr)
    
#     # Start from MSQ to LSQ
#     for i in range(num_qubits):
#         # Number of qubits to the right of the current qubit
#         k = num_qubits - i - 1
#         # Calculate probabilities for the current qubit being |0> and |1>
#         prob0 = 0
#         for j in range(0, 2**k, 2**k):
#             prob0 += np.abs(state_vector[j])**2
#         theta = 2 * np.arccos(np.sqrt(prob0))
#         qc.ry(theta, qr[i])
        
#         # Apply controlled rotations for entanglement
#         for j in range(1, 2**k):
#             binary = format(j, '0{}b'.format(k))
#             control_qubits = [qr[num_qubits - 1 - l] for l in range(k) if binary[-(l+1)] == '1']
#             if control_qubits:
#                 qc.cry(theta, control_qubits, qr[i])
    
#     return qc


from qiskit.circuit.library import RYGate
import numpy as np

def normalize_state(state_vector):
    """
    Normalize the state vector.
    """
    norm = np.linalg.norm(state_vector)
    if norm == 0:
        raise ValueError("State vector cannot be the zero vector.")
    return state_vector / norm

def get_bitstring(index, n_qubits):
    """
    Get the binary representation of an index with leading zeros to match n_qubits.
    """
    return format(index, f'0{n_qubits}b')

def compute_rotation_angles(state_vector, n_qubits):
    """
    Compute the rotation angles for each qubit.
    
    Returns a list of angles for each qubit.
    """
    angles = []
    for qubit in range(n_qubits):
        # Number of control bits processed so far
        controls = qubit
        step = 2 ** (controls + 1)
        theta_sum = 0
        for i in range(0, len(state_vector), step):
            block = state_vector[i:i + step]
            # Sum over half the block to get p0
            p0 = np.linalg.norm(block[:len(block)//2])**2
            theta = 2 * np.arccos(np.sqrt(p0))
            angles.append(theta)
    return angles

def state_preparation_mottonen(qc, qr, state_vector, n_qubits):
    """
    Prepare the quantum state using the Mottonen algorithm.
    
    Args:
        qc (QuantumCircuit): The quantum circuit.
        qr (QuantumRegister): The quantum register.
        state_vector (array_like): The desired state vector (must be normalized).
        n_qubits (int): Number of qubits.
    """
    # Iterate over qubits
    for qubit in range(n_qubits):
        # Compute the number of basis states controlled by previous qubits
        num_controls = qubit
        if num_controls == 0:
            # Apply Ry rotation to the first qubit based on the first half of amplitudes
            p0 = np.linalg.norm(state_vector[:2**(n_qubits - qubit -1)])**2
            theta = 2 * np.arccos(np.sqrt(p0))
            qc.ry(theta, qr[qubit])
        else:
            # For controlled rotations, iterate over all possible control states
            for control_state in range(2**num_controls):
                # Binary representation of the control state
                binary = format(control_state, f'0{num_controls}b')
                # Find indices where the first num_controls bits match the control_state
                indices = [i for i in range(len(state_vector)) if get_bitstring(i, n_qubits)[:num_controls] == binary]
                # Compute p0 for this control state
                p0 = np.linalg.norm(state_vector[indices[:len(indices)//2]])**2
                theta = 2 * np.arccos(np.sqrt(p0))
                
                # Apply controlled Ry
                # Determine the qubits that control this rotation
                control_qubits = [qr[i] for i in range(num_controls)]
                target_qubit = qr[qubit]
                
                # Create the controlled Ry gate
                ry_gate = RYGate(theta)
                if num_controls == 1:
                    qc.cry(theta, control_qubits[0], target_qubit)
                elif num_controls == 2:
                    qc.mcry(theta, control_qubits, target_qubit, None, mode='basic')
                elif num_controls == 3:
                    qc.mcry(theta, control_qubits, target_qubit, None, mode='basic')
                else:
                    raise NotImplementedError("This implementation supports up to 3 qubits.")
    
    # Optional: Reset the state if necessary

def state_preparation_circuit(state_vector):
    """
    Constructs a quantum circuit to prepare the given state vector using the Mottonen algorithm.
    
    Args:
        state_vector (array_like): The desired state vector (must be normalized).
        
    Returns:
        QuantumCircuit: The circuit that prepares the state.
    """
    state_vector = np.array(state_vector, dtype=complex)
    state_vector = normalize_state(state_vector)
    n_qubits = int(np.log2(len(state_vector)))
    if 2**n_qubits != len(state_vector):
        raise ValueError("Length of state vector must be a power of 2.")
    
    qr = QuantumRegister(n_qubits)
    qc = QuantumCircuit(qr)
    
    # Apply the Mottonen state preparation
    state_preparation_mottonen(qc, qr, state_vector, n_qubits)
    
    return qc.reverse_bits()

In [28]:
n = 2

rng = np.random.default_rng(932282547999982709834282302141)
psi = rng.random(2**n)

# psi = np.array([0.5, 0.5])
psi = psi / np.linalg.norm(psi, ord=2)
print(psi)
mycirc = state_preparation_circuit(psi)
mycirc_trans = transpile(mycirc, basis_gates=['u1', 'u2', 'u3', 'cx'])
print(Statevector(mycirc).data.real)
mycirc_trans.draw()

[0.26468034 0.34587318 0.57232196 0.69481195]
[ 0.43552719  0.         -0.57548341  0.69219572]


┌────────────────┐┌───┐┌─────────────────┐┌───┐┌─────────────────┐┌───┐»
q52_0: ┤ U3(1.3029,0,0) ├┤ X ├┤ U3(-1.3029,0,0) ├┤ X ├┤ U3(0.96146,0,0) ├┤ X ├»
       ├────────────────┤└─┬─┘└─────────────────┘└─┬─┘└─────────────────┘└─┬─┘»
q52_1: ┤ U3(2.2403,0,0) ├──■───────────────────────■───────────────────────■──»
       └────────────────┘                                                     »
«       ┌──────────────────┐┌───┐
«q52_0: ┤ U3(-0.96146,0,0) ├┤ X ├
«       └──────────────────┘└─┬─┘
«q52_1: ──────────────────────■──
«

In [29]:
qiscirc = QuantumCircuit(np.ceil(np.log(len(psi))))
qiscirc.initialize(psi)

qiscirc_trans = transpile(qiscirc, basis_gates=['u1', 'u2', 'u3', 'cx'])
qiscirc_trans.draw()

print("Answer", psi)
print()
print("My sol", Statevector(mycirc).data)
print()
print("Qiskit sol", Statevector(qiscirc).data)
print()
print()
print()
print("My circuit", mycirc_trans.count_ops())
print("Qiskit circuit", qiscirc_trans.count_ops())

Answer [0.26468034 0.34587318 0.57232196 0.69481195]

My sol [ 0.43552719+0.j  0.        +0.j -0.57548341+0.j  0.69219572+0.j]

Qiskit sol [0.26468034+0.j 0.34587318+0.j 0.57232196+0.j 0.69481195+0.j]



My circuit OrderedDict({'u3': 5, 'cx': 4})
Qiskit circuit OrderedDict({'u3': 3, 'reset': 2, 'cx': 1})
